## Utility Functions Load

In [ ]:
# %load utilities.py
def plot_distribution(data):
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats

    plt.figure()
    sns.distplot(data, fit=stats.norm);
    mean, std = stats.norm.fit(data)
    print('mean = {:.4f}\nstd = {:.4f}\nskewness = {:.4f}\nkurtosis = {:.4f}'
          .format(mean, std, data.skew(), data.kurtosis()))
    plt.figure()
    stats.probplot(data, plot=plt);

def describe_numerical(data):
    print('mean:', data.mean())
    print('median:', data.median())
    print('mode:', data.mode().values[0])
    print(data.describe())

def missing_value(data):
    print('missing value number:', data.isnull().sum())
    print('missing value percentage:', data.isnull().sum()/len(data))

def submit(test_X_og, pred_y):
    import pandas as pd

    submit = pd.DataFrame(data=[test_X_og.index, pred_y]).T
    submit.columns = ['PassengerId', 'Survived']
    submit = submit.astype('int32')
    submit.to_csv('submit.csv', index=False)

def gridsearchcv(model, param_grid, train_X, train_Y, dev_X, dev_Y):
    from sklearn.model_selection import (cross_val_score, GridSearchCV, KFold)
    from sklearn.metrics import accuracy_score

    model = GridSearchCV(model, param_grid=param_grid, scoring='accuracy',
                        cv=KFold(n_splits=5, shuffle=True, random_state=None))
    model.fit(train_X, train_Y)
    print('grid search best parameters:', model.best_params_)
    print('grid search best scores: {:.4f}'.format(model.best_score_))

    train_scores = cross_val_score(model, train_X, train_Y, scoring='accuracy',
                                cv=KFold(n_splits=5, shuffle=True, random_state=None))
    train_score = train_scores.mean()
    print('cv score: {:.4f}'.format(train_score))

    pred_y = model.predict(dev_X)
    dev_score = accuracy_score(dev_Y, pred_y)
    print('dev score: {:.4f}'.format(dev_score))

    return model

def plot_result(history):
    import matplotlib.pyplot as plt

    print('train set loss: {:.4f}'.format(history.history['loss'][-1]))
    print('dev set loss: {:.4f}'.format(history.history['val_loss'][-1]))
    print('train set accuracy: {:.4f}'.format(history.history['binary_accuracy'][-1]))
    print('dev set accuracy: {:.4f}'.format(history.history['val_binary_accuracy'][-1]))

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train Loss', 'Dev Loss'], loc='upper right')
    plt.show()

    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Loss Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train Accuracy', 'Dev Accuracy'], loc='upper right')
    plt.show()

def create_nn(train_X, train_Y, dev_X, dev_Y, l1, l2, lr, batch_size, epochs):
    from keras.callbacks import (History, EarlyStopping)
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras import losses
    from keras import metrics
    from keras import optimizers
    from keras import initializers
    from keras import regularizers

    model = Sequential()

    model.add(Dense(16, activation='relu',
              kernel_initializer=initializers.he_normal(seed=42),
              bias_initializer=initializers.Zeros(),
              kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
              bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))

    model.add(Dense(16, activation='relu',
              kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
              bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.Adam(lr=lr),
              loss=losses.binary_crossentropy,
              metrics = [metrics.binary_accuracy])

    history = History()
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(train_X, train_Y, validation_data=[dev_X, dev_Y], shuffle=True, verbose=0,
                batch_size=batch_size, epochs=epochs, callbacks=[history, early_stop])

    return model, history, early_stop


## Data Load

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
# from utilities import *
import matplotlib.pyplot as plt
%matplotlib inline
import re

from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from keras.callbacks import (History, EarlyStopping)
from keras.models import Sequential
from keras.layers import Dense
from keras import losses
from keras import metrics
from keras import optimizers
from keras import initializers
from keras import regularizers

import pickle
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload

train_og = pd.read_csv('../input/train.csv', index_col=0)
train_og = train_og.copy()
test_X_og = pd.read_csv('../input/test.csv', index_col=0)
test_X_og = test_X_og.copy()

print('train set shape:', train_og.shape)
print('test set shape:', test_X_og.shape)

train_X_og, train_Y_og = train_og[train_og.columns[1:]], train_og[train_og.columns[0]]

X_og = pd.concat([train_X_og, test_X_og])

## Feature Classification

In [ ]:
X_og = X_og.rename(columns={'Pclass' : 'class', 'Name' : 'full name', 
                           'Sex' : 'sex', 'Age' : 'age', 'SibSp' : 'family size 01', 
                           'Parch' : 'family size 02', 'Ticket' : 'ticket', 
                           'Fare' : 'fare', 'Cabin' : 'cabin class number', 'Embarked' : 'embarked'})

# print(X_og.columns)

categorical = ['class', 'full name', 'sex', 'ticket', 'cabin class number', 'embarked']
numerical = ['age', 'fare', 'family size 01', 'family size 02']

## Check Missing Values 

In [ ]:
# print('train_Y missing value:', train_Y_og.isnull().sum())
# print()
# print('X_og missing value number:')
# print((X_og.isnull().sum()).sort_values(ascending=False)[:10])
# print()
# print('X_og missing value percentage:')
# print((X_og.isnull().sum()/len(X_og)).sort_values(ascending=False)[:10])
X_fill = X_og.copy()

## 'cabin class number' Feature Missing Values

In [ ]:
# missing_value(X_og['cabin class number'])

X_fill['cabin class number'] = X_fill['cabin class number'].fillna('None')

## 'age' Feature Missing Values 

In [ ]:
# missing_value(X_og['age'])

# describe_numerical(X_og['age'])

X_fill['age'] = X_fill['age'].fillna(X_fill['age'].median())

## 'embarked' & 'fare' Feature Missing Values

In [ ]:
# missing_value(X_og['embarked'])

X_fill['embarked'].describe()

X_fill['embarked'] = X_fill['embarked'].fillna(X_fill['embarked'].describe()['top'])

# missing_value(X_og['fare'])

# describe_numerical(X_og['fare'])

X_fill['fare'] = X_fill['fare'].fillna(X_fill['fare'].median())

## Check Filled Missing Values

In [ ]:
# print('X_fill missing value number:')
# print((X_fill.isnull().sum()).sort_values(ascending=False)[:10])
# print('X_fill missing value percentage:')
# print((X_fill.isnull().sum()/len(X_fill)).sort_values(ascending=False)[:10])

## Numerical Feature Skewness & Kurtosis

In [ ]:
# for i in numerical: print(i+' skewness:', X_og[i].skew())

X_fill['fare'] = np.log1p(X_fill['fare'])

# print('fare skewness:', X_og['fare'].skew())

## Drop 'ticket' Feature

In [ ]:
X_fill = X_fill.drop('ticket', axis=1)

## Add Feature 'cabin class' & Drop 'cabin class number' Feature

In [ ]:
X_fill['cabin class'] = list(X_fill['cabin class number'].str[0])

X_fill = X_fill.drop('cabin class number', axis=1)

## Add Feature 'family size' & Drop 'family size 01/02' Features

In [ ]:
X_fill['family size'] = X_fill['family size 01'] + X_fill['family size 02'] + 1

X_fill = X_fill.drop(['family size 01', 'family size 02'], axis=1)

## Add Feature 'alone'

In [ ]:
X_fill['alone'] = np.where(X_fill['family size'] == 1, 1, 0)

## Add Feature 'honorific' & Drop 'full name' Feature

In [ ]:
honorific = [i[0] for i in X_fill['full name'].str.split(', ', expand=True)[1].str.split('.')]

X_fill['honorific'] = honorific

X_fill = X_fill.drop('full name', axis=1)

## Feature Value Transformation

In [ ]:
X_pre_transform = X_fill.copy()

# print(X_pre_transform.columns)

nominal = ['sex', 'alone', 'honorific']
ordinal = ['class', 'embarked', 'cabin class']
discrete = ['family size']
continuous = ['age', 'fare']

mapper = DataFrameMapper([
    (nominal, preprocessing.OneHotEncoder(sparse=False)), 
    (ordinal, preprocessing.OrdinalEncoder()), 
    (discrete, preprocessing.LabelEncoder()), 
    (continuous, preprocessing.StandardScaler())], 
    df_out=True)

X = mapper.fit_transform(X_pre_transform)

# print('X feature set shape:', X.shape)

## Create Train, Dev, Test Sets

In [ ]:
train_X = X.loc[train_X_og.index].values
test_X = X.loc[test_X_og.index].values

train_Y = train_Y_og.values

train_X, dev_X, train_Y, dev_Y = train_test_split(train_X, train_Y, random_state=42)

## Model - Neural Network

In [ ]:
def create_nn(train_X, train_Y, dev_X, dev_Y, l1, l2, lr, batch_size, epochs):
    from keras.callbacks import (History, EarlyStopping)
    from keras.models import Sequential
    from keras.layers import Dense, Dropout
    from keras import losses
    from keras import metrics
    from keras import optimizers
    from keras import initializers
    from keras import regularizers

    model = Sequential()

    model.add(Dense(16, activation='relu',
              kernel_initializer=initializers.he_normal(seed=42),
              bias_initializer=initializers.Zeros(),
              kernel_regularizer=regularizers.l1_l2(l1=l1, l2=l2),
              bias_regularizer=regularizers.l1_l2(l1=l1, l2=l2)))

    model.add(Dropout(rate=0.5, seed=42))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.Adam(lr=lr),
              loss=losses.binary_crossentropy,
              metrics = [metrics.binary_accuracy])

    history = History()
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(train_X, train_Y, validation_data=[dev_X, dev_Y], shuffle=True, verbose=0, 
              batch_size=batch_size, epochs=epochs, callbacks=[history, early_stop])

    return model, history, early_stop

In [ ]:
batch_size = 32
epochs = 1000
lr = 0.0001
l1 = 0.01
l2 = 0.01

In [ ]:
model_nn, history, early_stop = create_nn(train_X, train_Y, dev_X, dev_Y, l1, l2, lr, batch_size, epochs)

plot_result(history)

## Predict

In [ ]:
pickle.dump(model_nn, open('model_nn.sav', 'wb'))
model_nn_load = pickle.load(open('model_nn.sav', 'rb'))
y_nn = model_nn_load.predict(test_X)
y_nn_save = pd.DataFrame(data=[test_X_og.index, y_nn]).T
y_nn_save.columns = ['PassengerId', 'Survived']
y_nn_save = y_nn_save.astype('int32')
y_nn_save.to_csv('y_nn.csv', index=False)